In [1]:
from collections import Counter
from numpy import*
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import timeit
from numba import cuda
import warnings
warnings.filterwarnings("ignore")
import numba
from numba import jit, config, threading_layer,  prange, njit
import sys
import os
#Importing the required modules
import numpy as np
from scipy.stats import mode

In [2]:
Dataset = pd.read_csv('NSE-TATAGLOBAL11.csv')
Dataset

,Date,St Date,Diff,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
0,10/08/18,01/01/13,2106,208.00,222.25,206.85,216.00,215.15,4642146,10062.83
1,10/05/18,01/01/13,2103,217.00,218.60,205.90,210.25,209.20,3519515,7407.06
2,10/04/18,01/01/13,2102,223.50,227.80,216.15,217.25,218.20,1728786,3815.79
3,10/03/18,01/01/13,2101,230.00,237.50,225.75,226.45,227.60,1708590,3960.27
4,10/01/18,01/01/13,2099,234.55,234.60,221.05,230.30,230.90,1534749,3486.05
...,...,...,...,...,...,...,...,...,...,...
1230,10/14/13,01/01/13,286,160.85,161.45,157.70,159.30,159.45,1281419,2039.09
1231,10/11/13,01/01/13,283,161.15,163.45,159.00,159.80,160.05,1880046,3030.76
1232,10/10/13,01/01/13,282,156.00,160.80,155.85,160.30,160.15,3124853,4978.80
1233,10/09/13,01/01/13,281,155.70,158.20,154.15,155.30,155.55,2049580,3204.49


In [3]:
random.seed(0)

In [4]:
X = Dataset.iloc[:,2:7].values
Y = Dataset.iloc[:,7:8].values

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.5, random_state = 0)

In [6]:
from sklearn.preprocessing import StandardScaler
SC = StandardScaler()
X_train_KNN = SC.fit_transform(X_train)
X_test_KNN = SC.transform(X_test)

from sklearn.neighbors import KNeighborsRegressor

In [7]:

#Importing the required modules
import numpy as np
from scipy.stats import mode
 
#Euclidean Distance
def eucledian(p1,p2):
    dist = np.sqrt(np.sum((p1-p2)**2))
    return dist
 
#Function to calculate KNN
def predict(x_train, y , x_input, k):
    op_labels = []
     
    #Loop through the Datapoints to be classified
    for item in x_input: 
         
        #Array to store distances
        point_dist = []
         
        #Loop through each training Data
        for j in range(len(x_train)): 
            distances = eucledian(np.array(x_train[j,:]) , item) 
            #Calculating the distance
            point_dist.append(distances) 
        point_dist = np.array(point_dist) 
         
        #Sorting the array while preserving the index
        #Keeping the first K datapoints
        dist = np.argsort(point_dist)[:k] 
         
        #Labels of the K datapoints from above
        labels = y[dist]
         
        #Majority voting
        lab = mode(labels) 
        lab = lab.mode[0]
        op_labels.append(lab)
 
    return op_labels
start = timeit.default_timer()
y_pred = predict(X_train_KNN,Y_train,X_test_KNN , 2)
MSE = square(subtract(Y_test,y_pred)).mean() 
RMSE = sqrt(MSE)
stop = timeit.default_timer()
print('The RSME error is ',RMSE)
print('Time for serial run = ', stop - start)

The RSME error is  2.117084534345841
Time for serial run =  3.5574111999999998


In [8]:
config.THREADING_LAYER = 'omp'
config.NUMBA_NUM_THREADS=2
@numba.njit(parallel = True)
def eucledian(p1,p2):
    dist = np.sqrt(np.sum((p1-p2)**2))
    return dist
 

def predict(x_train, y , x_input, k):
    op_labels = []
     
    #Loop through the Datapoints to be classified
    for item in x_input: 
         
        #Array to store distances
        point_dist = []
         
        #Loop through each training Data
        for j in range(len(x_train)): 
            distances = eucledian(np.array(x_train[j,:]) , item) 
            #Calculating the distance
            point_dist.append(distances) 
        point_dist = np.array(point_dist) 
         
        #Sorting the array while preserving the index
        #Keeping the first K datapoints
        dist = np.argsort(point_dist)[:k] 
         
        #Labels of the K datapoints from above
        labels = y[dist]
         
        #Majority voting
        lab = mode(labels) 
        lab = lab.mode[0]
        op_labels.append(lab)

    return op_labels 
def _pred(p):
    p = predict(X_train_KNN,Y_train,X_test_KNN , 2)
    MSE = square(subtract(Y_test,p)).mean() 
    RMSE = sqrt(MSE)
    return RMSE
start = timeit.default_timer()
M = _pred(X_train_KNN)
stop = timeit.default_timer()
print('The RSME error is ', M)
print('Time after parallelisation = ', stop - start)

The RSME error is  2.117084534345841
Time after parallelisation =  6.139081599999999
